In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np

hrsWave = pd.read_csv("../Data/hrsWaveCleaned.csv")

n = 1000
hhidpn = np.random.choice(hrsWave["HHIDPN"].unique(), size = n)
idx = hrsWave["HHIDPN"].isin(hhidpn)
df = hrsWave.loc[idx, :]

n_trials = 20
# 1. Define the Binomial family and logit link
# The `Binomial` family in statsmodels assumes endog is proportions (e.g., successes/n_trials)
# or a 2-column array where col 0 is successes and col 1 is failures.
# Since your score is 0-20, you should pass it as a two-column array: [recall_score, 20 - recall_score]


# Create a 2-column array for endog if using the formula API with a non-standard endog
# For GEE with Binomial, endog usually expects a proportion (successes/n_trials) or a (successes, total_trials) tuple/array
# If using `smf.gee`, it's often more straightforward to define `endog` as proportion.
df['RwRecProp'] = df['RwTR20'] / n_trials

C:\Users\86139\AppData\Local\Temp\ipykernel_11492\184860265.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RwRecProp'] = df['RwTR20'] / n_trials


## 1. Fitting the Base Model

In [13]:
def fitBaseModel(formula, groups, df, covStruct, family):
    model = smf.gee(formula, groups=groups, data=df,
                    cov_struct=covStruct,
                    family=family).fit(cov_type = "robust")
    
    print(model.summary())
    

    return model

# 2. Define the exchangeable correlation structure
exchangeable_corr = sm.cov_struct.Exchangeable()
autoregress_corr = sm.cov_struct.Autoregressive()
indep_corr = sm.cov_struct.Independence()


# 3. Specify Full Model Formula
formulaBase = "RwRecProp ~ RwAGEM_B * C(RwJOCCSD, Treatment(reference='Retired'))"

# 4. Fit the model with robust covariance
resultsExch = fitBaseModel(formulaBase, "HHIDPN", df, exchangeable_corr, sm.families.Binomial())
resultsAR = fitBaseModel(formulaBase, "HHIDPN", df, autoregress_corr, sm.families.Binomial())
resultsIndep = fitBaseModel(formulaBase, "HHIDPN", df, indep_corr, sm.families.Binomial())

C:\Users\86139\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\genmod\cov_struct.py:796: FutureWarning: grid=True will become default in a future version
  warnings.warn(


                               GEE Regression Results                              
Dep. Variable:                   RwRecProp   No. Observations:                 4069
Model:                                 GEE   No. clusters:                      906
Method:                        Generalized   Min. cluster size:                   1
                      Estimating Equations   Max. cluster size:                  11
Family:                           Binomial   Mean cluster size:                 4.5
Dependence structure:         Exchangeable   Num. iterations:                     9
Date:                     Fri, 27 Jun 2025   Scale:                           1.000
Covariance type:                    robust   Time:                         19:54:29
                                                                                               coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------

## 2. Variable Selection by QIC:

In [ ]:
# Calculate QIC for each model as a measure of goodness of fit
def calc_qic(model_result):
    # QIC = -2 * quasi-likelihood + 2 * trace(V_hat^-1 * V_model)
    # statsmodels does not provide QIC directly, but we can use qic() if available
    try:
        return model_result.qic()[0]
    except AttributeError:
        return np.nan

qic_exch = calc_qic(resultsExch)
qic_ar = calc_qic(resultsAR)
qic_unstruct = calc_qic(resultsUnstruct)

print(f"QIC (Exchangeable): {qic_exch}")
print(f"QIC (Autoregressive): {qic_ar}")
print(f"QIC (Unstructured): {qic_unstruct}")

# The base model with independence covariance structure yields the lowest QIC, marking the best performing base model.

C:\Users\86139\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\genmod\generalized_estimating_equations.py:1934: UserWarning: QIC values obtained using scale=None are not appropriate for comparing models
  warnings.warn("QIC values obtained using scale=None are not "


QIC (Exchangeable): 551.7757181441755
QIC (Autoregressive): 555.1874580582319
QIC (Unstructured): 541.1355708237093


In [15]:
import gc
gc.collect()

468

In [17]:
## Choose the control variables
controlVars = ['RwWORK', 'RwJHOURS', 'RwWGIHR',
       'RwJPHYS', 'RwJLIFT', 'RwJSTRES', 'RwJSTOOP', 'RwJSIGHT', 'RwCENREG',
       'RwMSTAT', 'RwLIVBRO', 'RwHIBP', 'RwDIAB', 'RwCANCR',
       'RwLUNG', 'RwHEART', 'RwSTROK', 'RwPSYCH', 'RwVIGACT', 'RwSMOKEV',
       'RwDRINK', 'RwPhyLim', 'RwCogLim', 'RwAnyCogImp', 'RwLOST',
       'RwWANDER', 'RwHALUC', 'RwALONE', 'HwATOTB', 'HwADEBT', 'HwACHCK',
       'HwAMRTB', 'HwITOT', 'RAGENDER', 'RARACEM', 'RAEDYRS',
       'RAEVBRN']

baseFormula = "RwRecProp ~ RwAGEM_B * C(RwJOCCSD, Treatment(reference='Retired'))"
fullFormula = baseFormula + ' + ' + ' + '.join(controlVars)

In [18]:
import re
import numpy as np
def stepwise_selection_qic(data, groups, cov_struct, family, start_formula, end_formula, verbose=True, qic_threshold=0):
    """
    Perform stepwise (forward and backward) feature selection based on QIC for GEE models.

    Parameters:
        data: pandas.DataFrame
            The dataset containing all variables used in the formulas.
        groups: array-like
            Grouping variable for GEE (e.g., subject or cluster IDs).
        cov_struct: statsmodels.genmod.cov_struct.CovStruct
            Covariance structure for GEE (e.g., Exchangeable, Autoregressive).
        family: statsmodels.genmod.families.Family
            The family object for GEE (e.g., Gaussian, Binomial).
        start_formula: str
            The starting model formula (patsy syntax).
        end_formula: str
            The full model formula (patsy syntax, includes all candidate variables).
        verbose: bool, optional
            If True, prints progress at each step.
        qic_threshold: float, optional
            Minimum QIC improvement required to continue selection.

    Returns:
        best_formula: str, formula of the best model found
        best_result: fitted GEE result object
        history: list of (formula, QIC)
    """
    import statsmodels.formula.api as smf

    def get_terms(formula):
        rhs = formula.split('~')[1]
        terms = [t.strip() for t in re.split(r'\s*\+\s*', rhs) if t.strip() != '']
        terms = [t for t in terms if t != '1']
        return set(terms)

    def build_formula(lhs, terms):
        if not terms:
            return f"{lhs} ~ 1"
        return f"{lhs} ~ {' + '.join(sorted(terms))}"

    def calc_qic(result):
        try:
            return result.qic(scale = 1)[0]
        except Exception:
            return np.nan

    lhs = start_formula.split('~')[0].strip()
    start_terms = get_terms(start_formula)
    end_terms = get_terms(end_formula)
    current_terms = set(start_terms)
    history = []

    # Fit initial model
    current_formula = build_formula(lhs, current_terms)
    model = smf.gee(current_formula, groups=groups, data=data, cov_struct=cov_struct, family=family)
    result = model.fit(cov_type="robust")
    best_qic = calc_qic(result)
    best_formula = current_formula
    best_result = result
    history.append((current_formula, best_qic))

    improved = True
    while improved:
        improved = False
        # Forward step
        qic_candidates = []
        formulas = []
        term_changes = []
        for term in sorted(end_terms - current_terms):
            new_terms = current_terms | {term}
            formula = build_formula(lhs, new_terms)
            try:
                model = smf.gee(formula, groups=groups, data=data, cov_struct=cov_struct, family=family)
                result = model.fit(cov_type="robust")
                qic = calc_qic(result)
            except Exception:
                qic = np.nan
            qic_candidates.append(qic)
            formulas.append(formula)
            term_changes.append(('add', term))

        # Backward step
        for term in sorted(current_terms - start_terms):
            new_terms = current_terms - {term}
            formula = build_formula(lhs, new_terms)
            try:
                model = smf.gee(formula, groups=groups, data=data, cov_struct=cov_struct, family=family)
                result = model.fit(cov_type="robust")
                qic = calc_qic(result)
            except Exception:
                qic = np.nan
            qic_candidates.append(qic)
            formulas.append(formula)
            term_changes.append(('remove', term))
        
        # print(qic_candidates)
        if qic_candidates:
            min_idx = np.nanargmin(qic_candidates)
            min_qic = qic_candidates[min_idx]
            if (best_qic - min_qic) > qic_threshold:
                improved = True
                best_qic = min_qic
                best_formula = formulas[min_idx]
                action, term = term_changes[min_idx]
                if action == 'add':
                    current_terms.add(term)
                elif action == 'remove':
                    current_terms.remove(term)
                model = smf.gee(best_formula, groups=groups, data=data, cov_struct=cov_struct, family=family)
                best_result = model.fit(cov_type="robust")
                history.append((best_formula, best_qic))
                if verbose:
                    print(f"Step: {action}, QIC: {best_qic:.2f}, Formula: {best_formula}")
            else:
                if verbose:
                    print("No QIC improvement above threshold, stopping.")
        else:
            if verbose:
                print("No candidates left, stopping.")

    return best_formula, best_result, history

In [19]:
bestExchForm, bestExchResult, ExchHistory =\
    stepwise_selection_qic(df, df["HHIDPN"], exchangeable_corr, 
                        sm.families.Binomial(), 
                        baseFormula, fullFormula, 
                        verbose=False, 
                        qic_threshold=10)

bestARForm, bestARResult, ARHistory =\
    stepwise_selection_qic(df, df["HHIDPN"], autoregress_corr, 
                        sm.families.Binomial(), 
                        baseFormula, fullFormula, 
                        verbose=False, 
                        qic_threshold=10)

bestIndepForm, bestIndepResult, IndepHistory =\
    stepwise_selection_qic(df, df["HHIDPN"], indep_corr, 
                        sm.families.Binomial(), 
                        baseFormula, fullFormula, 
                        verbose=False, 
                        qic_threshold=10)

In [21]:
print("Best formula of Exchangeble Covariance:", bestExchForm)
print("Best formula of AR(1) Covariance:", bestARForm)
print("Best formula of Indepedence Covariance:", bestIndepForm)

print("Best qic of Exchangeble Covariance:", ExchHistory[-1][1])
print("Best qic of AR(1) Covariance:", ARHistory[-1][1])
print("Best qic of Indepedence Covariance:", IndepHistory[-1][1])

Best formula of Exchangeble Covariance: RwRecProp ~ RwAGEM_B * C(RwJOCCSD, Treatment(reference='Retired')) + RwJLIFT + RwSMOKEV
Best formula of AR(1) Covariance: RwRecProp ~ RAEDYRS + RAGENDER + RwAGEM_B * C(RwJOCCSD, Treatment(reference='Retired'))
Best formula of Indepedence Covariance: RwRecProp ~ RwAGEM_B * C(RwJOCCSD, Treatment(reference='Retired')) + RwWGIHR
Best qic of Exchangeble Covariance: -408.78824872787925
Best qic of AR(1) Covariance: 476.7440560651559
Best qic of Indepedence Covariance: 87.33868652831293


In [27]:
print("Best formula of Exchangeble Covariance:", bestExchResult.summary().tables[1])
print("Best formula of AR(1) Covariance:", bestARResult.summary().tables[1])
print("Best formula of Indepedence Covariance:", bestIndepResult.summary().tables[1])

Best formula of Exchangeble Covariance: ============================================================================================================================================================
                                                                                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                    0.3182        nan        nan        nan         nan         nan
C(RwJOCCSD, Treatment(reference='Retired'))[T.Farming/Forestry/Fishing]                     -0.5736        nan        nan        nan         nan         nan
C(RwJOCCSD, Treatment(reference='Retired'))[T.Food/Personal/Service]                         1.0898        nan        nan        nan         nan         nan
C(RwJOCCSD, Treatm

## 3. Automated Hypothesis Testing

In [ ]:
import re

# ## A List of parameters
param_names = bestExchResult.params.index.tolist()

# ## Create joint hypothesis tests for "No interactions"
interaction_pattern = re.compile(r"RwAGEM_B:C\(RwJOCCSD,")
interaction_indices = [i for i, name in enumerate(param_names) if interaction_pattern.search(name)]
interaction_matrix = np.zeros((len(interaction_indices), len(param_names)))
for row, idx in enumerate(interaction_indices):
    interaction_matrix[row, idx] = 1

## Create joint hypothesis tests for "No main effect"
main_pattern = re.compile(r"^C\(RwJOCCSD,")
main_indices = [i for i, name in enumerate(param_names) if main_pattern.search(name)]
main_matrix = np.zeros((len(main_indices), len(param_names)))
for row, idx in enumerate(main_indices):
    main_matrix[row, idx] = 1

# ## Create the hypothesis test for STEM superior
stem_matrix = np.zeros((1, len(param_names)))
hypo_coef = [-1, -1, -1, -1, -1, 7, -1, -1]
for j in list(range(0, len(main_indices))):
    stem_matrix[0, main_indices[j]] = hypo_coef[j]

## Create the hypothesis test for Management superior
mgmt_matrix = np.zeros((1, len(param_names)))
hypo_coef = [-1, -1, -1, -1, 7, -1, -1, -1]
for j in list(range(0, len(main_indices))):
    mgmt_matrix[0, main_indices[j]] = hypo_coef[j]


## Create the hypothesis test for food inferior
food_matrix = np.zeros((1, len(param_names)))
hypo_coef = [-1, 7, -1, -1, -1, -1, -1, -1]
for j in list(range(0, len(main_indices))):
    food_matrix[0, main_indices[j]] = hypo_coef[j]

## Create the hypothesis test for farm inferior
farm_matrix = np.zeros((1, len(param_names)))
hypo_coef = [7, -1, -1, -1, -1, -1, -1, -1]
for j in list(range(0, len(main_indices))):
    farm_matrix[0, main_indices[j]] = hypo_coef[j]




In [ ]:
def testSummary(r_matrix, model):
    wald_res = model.wald_test(r_matrix)
    print("Statistic:", wald_res.statistic[0,0])
    print("Degrees of freedom:", wald_res.df_denom)
    print("p-value:", wald_res.pvalue)
    print("Distribution:", wald_res.distribution)

def serialTest(model, varPattern):
    '''
    Test multipe similar null hypothesis independently
    '''
    param_names = model.params.index.tolist()

    # Indices of interested parameters
    main_pattern = re.compile(varPattern)
    main_indices = [i for i, name in enumerate(param_names) if main_pattern.search(name)]

    l = len(main_indices)
    for j in list(range(0, l)):
        ## Hypothesis Coefficients
        hypo_coef = np.full((1, l), -1/l)
        hypo_coef[0, j] = 1

        ## Map hypo_coef to the parameter
        r_matrix = np.zeros((1, len(param_names)))

        # print(hypo_coef[0,1])
        # print(main_indices)
        # print(r_matrix)
        for k in list(range(0, len(main_indices))):
            r_matrix[0, main_indices[k]] = hypo_coef[0,k]

        ## Net effect estimate
        netEffect = (r_matrix @ np.array(model.params))[0]
        print(f"Net effect of {param_names[main_indices[j]]} is {netEffect}")
        
        ## Wald test 
        testSummary(r_matrix, model)
        print("\n\n")

    
# testSummary(interaction_matrix, bestARResult)
serialTest(bestARResult, "^C\(RwJOCCSD,")

Net effect of C(RwJOCCSD, Treatment(reference='Retired'))[T.Farming/Forestry/Fishing] is -2.0163154660991154
Statistic: 1.0024568761662067
Degrees of freedom: 1.0
p-value: 0.3167167451578227
Distribution: chi2



Net effect of C(RwJOCCSD, Treatment(reference='Retired'))[T.Food/Personal/Service] is 0.340752255156059
Statistic: 0.1871875010566847
Degrees of freedom: 1.0
p-value: 0.6652678176953666
Distribution: chi2



Net effect of C(RwJOCCSD, Treatment(reference='Retired'))[T.Healthcare] is -0.6261271487591735
Statistic: 0.457627835869103
Degrees of freedom: 1.0
p-value: 0.49873469890342914
Distribution: chi2



Net effect of C(RwJOCCSD, Treatment(reference='Retired'))[T.High Risk Occupations] is 2.9129773294032537
Statistic: 1.109016672128203
Degrees of freedom: 1.0
p-value: 0.2922958020921877
Distribution: chi2



Net effect of C(RwJOCCSD, Treatment(reference='Retired'))[T.Management/Clerical/Business] is -0.10738447069569762
Statistic: 0.029175358341196746
Degrees of freedom: 1.0
p-

C:\Users\86139\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\base\model.py:1912: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


In [ ]:
## Multiple tests on Interaction Terms
serialTest(bestARResult, "^RwAGEM_B:C\(RwJOCCSD,")

Net effect of RwAGEM_B:C(RwJOCCSD, Treatment(reference='Retired'))[T.Farming/Forestry/Fishing] is 0.035807682949691555
Statistic: 1.1138568383756848
Degrees of freedom: 1.0
p-value: 0.29124509153381184
Distribution: chi2



Net effect of RwAGEM_B:C(RwJOCCSD, Treatment(reference='Retired'))[T.Food/Personal/Service] is -0.006149995862626768
Statistic: 0.23417399573963538
Degrees of freedom: 1.0
p-value: 0.6284459975690349
Distribution: chi2



Net effect of RwAGEM_B:C(RwJOCCSD, Treatment(reference='Retired'))[T.Healthcare] is 0.008672970360212976
Statistic: 0.2950677957107403
Degrees of freedom: 1.0
p-value: 0.5869911207533771
Distribution: chi2



Net effect of RwAGEM_B:C(RwJOCCSD, Treatment(reference='Retired'))[T.High Risk Occupations] is -0.05018521275236207
Statistic: 1.1111731732317598
Degrees of freedom: 1.0
p-value: 0.29182706885164766
Distribution: chi2



Net effect of RwAGEM_B:C(RwJOCCSD, Treatment(reference='Retired'))[T.Management/Clerical/Business] is 0.0030246022191070523


In [ ]:
# Retrieve the covariance matrices of the fitted GEE models
# cov_exch = resultsExch.cov_params()
# cov_ar = resultsAR.cov_params()
# cov_unstruct = resultsUnstruct.cov_params()

# print("Covariance matrix (Exchangeable):\n", cov_exch)
# print("\nCovariance matrix (Autoregressive):\n", cov_ar)
# print("\nCovariance matrix (Unstructured):\n", cov_unstruct)